In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Define constants
IMG_SIZE = (48, 48)
BATCH_SIZE = 128
EPOCHS = 30
NUM_CLASSES = 7

# Load FER-2013 dataset
df = pd.read_csv('fer2013.csv')

# Process images and labels
X, y = [], []
for _, row in df.iterrows():
    pixels = np.array(row['pixels'].split(), dtype='float32').reshape(48, 48)
    X.append(np.stack((pixels,) * 3, axis=-1))  # Convert grayscale to RGB
    y.append(row['emotion'])

X = np.array(X) / 255.0  # Normalize images
y = np.array(y)

# Oversample minority classes
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)
X, y = ros.fit_resample(X.reshape(X.shape[0], -1), y)  # Reshape to 2D for oversampling
X = X.reshape(-1, 48, 48, 3)  # Reshape back

# Convert labels to one-hot encoding
y = to_categorical(y, num_classes=NUM_CLASSES)

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2
)
train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)

# Load VGG16 pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Fine-tune last 6 layers
for layer in base_model.layers[:-6]:
    layer.trainable = False

# Add custom classification head
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(NUM_CLASSES, activation='softmax', dtype='float32')(x)

# Create model
model = Model(inputs=base_model.input, outputs=out)

# Define Focal Loss
def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        y_pred = tf.keras.backend.clip(y_pred, 1e-7, 1 - 1e-7)
        loss = -y_true * (alpha * tf.keras.backend.pow(1 - y_pred, gamma) * tf.keras.backend.log(y_pred))
        return tf.keras.backend.sum(loss, axis=1)
    return loss

# Compile model with Focal Loss
model.compile(optimizer=Adam(learning_rate=0.0001), loss=focal_loss(), metrics=['accuracy'])

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Train model
model.fit(train_generator, validation_data=(X_val, y_val), epochs=EPOCHS, callbacks=[reduce_lr])

# Save model
model.save('fer_vgg16_oversampled_focal_loss.h5')
print("Model saved successfully!")


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


394/394 ━━━━━━━━━━━━━━━━━━━━ 61s 144ms/step - accuracy: 0.1803 - loss: 0.3579 - val_accuracy: 0.2420 - val_loss: 0.3172 - learning_rate: 1.0000e-04
Epoch 2/30
394/394 ━━━━━━━━━━━━━━━━━━━━ 58s 146ms/step - accuracy: 0.2455 - loss: 0.3193 - val_accuracy: 0.2934 - val_loss: 0.3136 - learning_rate: 1.0000e-04
Epoch 3/30
394/394 ━━━━━━━━━━━━━━━━━━━━ 81s 143ms/step - accuracy: 0.3175 - loss: 0.2946 - val_accuracy: 0.4508 - val_loss: 0.2345 - learning_rate: 1.0000e-04
Epoch 4/30
394/394 ━━━━━━━━━━━━━━━━━━━━ 58s 148ms/step - accuracy: 0.4222 - loss: 0.2461 - val_accuracy: 0.5092 - val_loss: 0.2064 - learning_rate: 1.0000e-04
Epoch 5/30
394/394 ━━━━━━━━━━━━━━━━━━━━ 59s 151ms/step - accuracy: 0.4859 - loss: 0.2171 - val_accuracy: 0.5480 - val_loss: 0.1810 - learning_rate: 1.0000e-04
Epoch 6/30
394/394 ━━━━━━━━━━━━━━━━━━━━ 53s 133ms/step - accuracy: 0.5225 - loss: 0.1975 - val_accuracy: 0.5649 - val_loss: 0.1705 - learning_rate: 1.0000e-04
Epoch 7/30
394/394 ━━━━━━━━━━━━━━━━━━━━ 53s 134ms/step - 

Model saved successfully!
